# Searching A Single Document - Rijkswaterstaat (RWS)

A test on how to load a PDF document and search it using the Perry AI Powered Document Analysis tool.

## Constructing The File Nodes

In [1]:
# Check if doc_data/RWS exists, give warning if not
import pathlib

def get_data_path():
    """Returns the path to the RWS data directory."""
    return pathlib.Path.cwd() / "doc_data" / "LIVE"

rws_path = get_data_path()
if not rws_path.exists():
    print("Warning: RWS data not found. Please create the directory and fill it with document data first.")

if not any(rws_path.iterdir()):
    print("Warning: RWS data not found. Please fill the directory with document data first.")

In [2]:
import os
import openai
from dotenv import load_dotenv
from llama_index import VectorStoreIndex, StorageContext, ServiceContext, load_index_from_storage
from llama_index.readers.base import BaseReader
from llama_index.schema import Document

def load_openai_api_key():
    """Load OpenAI API key from environment variable."""
    load_dotenv()
    openai.api_key = os.environ["OPENAI_API_KEY"]

def get_rws_file_paths(data_dir: pathlib.Path) -> list[pathlib.Path]:
    """Get file paths of RWS documents."""
    return [f for f in data_dir.iterdir() if f.is_file()]

def load_rws_documents(data_dir: pathlib.Path, loader: BaseReader) -> dict[str, list[Document]]:
    """Load RWS documents."""
    doc_set = {}
    for doc_path in get_rws_file_paths(data_dir):
        docs = loader.load_data(file=doc_path, split_documents=False)
        doc_set[doc_path.stem] = docs
    return doc_set
    
def create_vector_indexes(doc_set: dict[str, list[Document]]) -> dict[str, list[VectorStoreIndex]]:
    """Create and store vector indexes."""
    service_context = ServiceContext.from_defaults(chunk_size=512)
    vector_indexes = {}
    for doc_name, docs in doc_set.items():
        storage_context = StorageContext.from_defaults()
        vector_index = VectorStoreIndex.from_documents(
            docs,
            service_context=service_context,
            storage_context=storage_context
        )
        vector_indexes[doc_name] = vector_index
    return vector_indexes

def save_vector_indexes(vector_indexes: dict[str, list[VectorStoreIndex]]) -> None:
    """Save vector indexes."""
    for doc_name, vector_index in vector_indexes.items():
        vector_index.storage_context.persist(persist_dir=f'./storage/rws/{doc_name}')

def load_vector_indexes() -> dict[str, list[VectorStoreIndex]]:
    """Load vector indexes."""
    vector_indexes = {}
    for doc_name in get_rws_file_paths(rws_path):
        storage_context = StorageContext.from_defaults(persist_dir=f'./storage/rws/{doc_name.stem}')
        vector_index = load_index_from_storage(storage_context=storage_context)
        vector_indexes[doc_name.stem] = vector_index
    return vector_indexes


In [3]:
load_openai_api_key()

In [4]:
# Load in docs
from llama_index import download_loader
UnstructuredReader = download_loader("UnstructuredReader", refresh_cache=True)
loader = UnstructuredReader()
docs = load_rws_documents(rws_path, loader)

[nltk_data] Downloading package punkt to /home/bluegnome/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/bluegnome/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
vector_indexes_new = create_vector_indexes(docs)
save_vector_indexes(vector_indexes_new)

In [6]:
vector_indexes = load_vector_indexes()

In [7]:
vector_indexes_new.keys()

dict_keys(['Vraagspecificatie Projectmanagementspecificatie BOC 1.0'])

In [8]:
vector_indexes.keys()

dict_keys(['Vraagspecificatie Projectmanagementspecificatie BOC 1.0'])

# Create Langchain Tooling

Create langchain tooling from the loaded indexes.

In [9]:
from langchain.chains.conversation.memory import ConversationBufferMemory

from llama_index.langchain_helpers.agents import LlamaToolkit, create_llama_chat_agent, IndexToolConfig
from llama_index.bridge.langchain import AgentExecutor

from langchain.chat_models import ChatOpenAI


def create_vector_index_tool_configs(vector_indexes: dict[str, list[VectorStoreIndex]]) -> dict[str, IndexToolConfig]:
    """Create vector index tool configs."""
    index_tool_configs = {}
    for doc_name, vector_index in vector_indexes.items():
        query_engine = vector_index.as_query_engine(
            similarity_top_k=3,
        )
        index_tool_configs[doc_name] = IndexToolConfig(
            query_engine=query_engine,
            name=f"Vector Index {doc_name}",
            description=f"Aanbestedingsleidraad voor rijkswaterstaat opdrachten",
            tool_kwargs={"return_direct": True}
        )
    return index_tool_configs

def create_langchain_toolkit(index_tool_configs: dict[str, IndexToolConfig]) -> LlamaToolkit:
    """Create LangChain toolkit."""
    return LlamaToolkit(index_configs= list(index_tool_configs.values()))

def create_langchain_chat_agent(toolkit: LlamaToolkit) -> AgentExecutor:
    """Create LangChain chat agent."""
    memory = ConversationBufferMemory(memory_key="chat_history")
    llm=ChatOpenAI(temperature=0, model_name="gpt-4")
    return create_llama_chat_agent(
        toolkit,
        llm,
        memory=memory,
        verbose=True
    )


In [10]:
toolkit = create_langchain_toolkit(create_vector_index_tool_configs(vector_indexes))
agent = create_langchain_chat_agent(toolkit)

# Ask Questions For Testing

In [12]:
agent.run(input="Welke maatregelen staan benoemd in Vraagspecificatie Projectmanagementspecificatie BOC 1.0.pdf dragen bij aan het realiseren van criterium 'Optimale samenwerking tussen OG - ON'")



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Vector Index Vraagspecificatie Projectmanagementspecificatie BOC 1.0
Action Input: 'Optimale samenwerking tussen OG - ON' maatregelen
Observation: 
Optimale samenwerking tussen de Opdrachtgever (OG) en de Opdrachtnemer (ON) maatregelen kunnen worden gevonden in het organiseren van periodieke besprekingen in het kader van en ten behoeve van de samenwerkingsmonitor, het organiseren van een Project-Start-Up (PSU) en Project-Follow-Up's (PFU's), het organiseren van een overleg waarbij de voortgangsrapportage wordt besproken, het organiseren van een overleg waarin veiligheidsdocumenten formeel door de Opdrachtgever worden overgedragen en integrale veiligheidsrisico's en beheersmaatregelen worden besproken, het organiseren van een overleg waarin de door de Opdrachtnemer geprognosticeerde CO2-footprint, zijn projectdoelstelling ten aanzien van de kwantitatieve CO2-reductie, en de door hem daartoe voorgenomen maatregelen 

"\nOptimale samenwerking tussen de Opdrachtgever (OG) en de Opdrachtnemer (ON) maatregelen kunnen worden gevonden in het organiseren van periodieke besprekingen in het kader van en ten behoeve van de samenwerkingsmonitor, het organiseren van een Project-Start-Up (PSU) en Project-Follow-Up's (PFU's), het organiseren van een overleg waarbij de voortgangsrapportage wordt besproken, het organiseren van een overleg waarin veiligheidsdocumenten formeel door de Opdrachtgever worden overgedragen en integrale veiligheidsrisico's en beheersmaatregelen worden besproken, het organiseren van een overleg waarin de door de Opdrachtnemer geprognosticeerde CO2-footprint, zijn projectdoelstelling ten aanzien van de kwantitatieve CO2-reductie, en de door hem daartoe voorgenomen maatregelen worden besproken, en het organiseren van een overleg waarin het Gezamenlijk managen van de onderhoudsbehoefte wordt besproken. Daarnaast kunnen maatregelen worden genomen om te voldoen aan de Wet arbeid vreemdelingen (

In [13]:
agent.run(input="Kun je dat samenvatten in b1 taalniveau?")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Zeker, ik zal het eenvoudiger maken. Er zijn verschillende manieren om de samenwerking tussen de opdrachtgever en de opdrachtnemer te verbeteren. Dit kan door regelmatig te overleggen over de voortgang van het project, veiligheidsdocumenten en plannen om de CO2-uitstoot te verminderen. Ook is het belangrijk om goed te communiceren met alle betrokken partijen en om te voldoen aan de wetten en regels die van toepassing zijn op het werk.

> Finished chain.


'Zeker, ik zal het eenvoudiger maken. Er zijn verschillende manieren om de samenwerking tussen de opdrachtgever en de opdrachtnemer te verbeteren. Dit kan door regelmatig te overleggen over de voortgang van het project, veiligheidsdocumenten en plannen om de CO2-uitstoot te verminderen. Ook is het belangrijk om goed te communiceren met alle betrokken partijen en om te voldoen aan de wetten en regels die van toepassing zijn op het werk.'

In [14]:
agent.run(input="Welke maatregelen buiten diegene die net zijn benoemd dragen bij aan het realiseren van criterium 'Optimale samenwerking tussen OG - ON'?, zodanig dat dit een positieve bijdrage levert aan de onderhoudsbehoefte van het areaal?")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Naast de eerder genoemde maatregelen, zijn er nog andere manieren om de samenwerking tussen de opdrachtgever en de opdrachtnemer te verbeteren en zo bij te dragen aan de onderhoudsbehoefte van het areaal. 

1. Heldere communicatie: Zorg ervoor dat beide partijen duidelijk zijn over hun verwachtingen en doelen. Dit kan misverstanden voorkomen en zorgen voor een soepelere samenwerking.

2. Regelmatige updates: Houd elkaar op de hoogte van de voortgang van het project. Dit kan helpen om problemen vroegtijdig te identificeren en aan te pakken.

3. Gezamenlijke planning: Werk samen aan de planning van het onderhoud. Dit kan helpen om ervoor te zorgen dat alle noodzakelijke werkzaamheden worden uitgevoerd en dat er geen belangrijke taken worden gemist.

4. Training en ontwikkeling: Zorg ervoor dat alle teamleden de nodige training en ontwikkelingsmogelijkheden krijgen. Dit kan helpen om ervoor te zorgen dat iedereen de vaard

'Naast de eerder genoemde maatregelen, zijn er nog andere manieren om de samenwerking tussen de opdrachtgever en de opdrachtnemer te verbeteren en zo bij te dragen aan de onderhoudsbehoefte van het areaal. \n\n1. Heldere communicatie: Zorg ervoor dat beide partijen duidelijk zijn over hun verwachtingen en doelen. Dit kan misverstanden voorkomen en zorgen voor een soepelere samenwerking.\n\n2. Regelmatige updates: Houd elkaar op de hoogte van de voortgang van het project. Dit kan helpen om problemen vroegtijdig te identificeren en aan te pakken.\n\n3. Gezamenlijke planning: Werk samen aan de planning van het onderhoud. Dit kan helpen om ervoor te zorgen dat alle noodzakelijke werkzaamheden worden uitgevoerd en dat er geen belangrijke taken worden gemist.\n\n4. Training en ontwikkeling: Zorg ervoor dat alle teamleden de nodige training en ontwikkelingsmogelijkheden krijgen. Dit kan helpen om ervoor te zorgen dat iedereen de vaardigheden en kennis heeft die nodig zijn om hun taken effecti

In [15]:
agent.run(input="Dit is een generiek antwoord, kun je dit specifieker maken voor dit project? Het is ok om te speculeren, maar benoem dat expliciet.")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Natuurlijk, ik zal proberen specifieker te zijn, hoewel ik moet benadrukken dat dit speculatief is omdat ik geen specifieke details over het project heb.

1. Gezamenlijke risicobeoordeling: Voor dit specifieke project kan het nuttig zijn om gezamenlijk een risicobeoordeling uit te voeren. Dit kan helpen om potentiële problemen te identificeren en plannen te maken om deze aan te pakken, wat kan bijdragen aan de onderhoudsbehoefte van het areaal.

2. Technologie gebruiken: Afhankelijk van de aard van het project, kan het gebruik van bepaalde technologieën, zoals geavanceerde monitoring- en onderhoudssystemen, helpen om de onderhoudsbehoefte te verminderen en de samenwerking tussen de opdrachtgever en de opdrachtnemer te verbeteren.

3. Regelmatige sitebezoeken: Voor dit project kan het nuttig zijn om regelmatige sitebezoeken te organiseren. Dit kan zowel de opdrachtgever als de opdrachtnemer een beter inzicht geven in de

'Natuurlijk, ik zal proberen specifieker te zijn, hoewel ik moet benadrukken dat dit speculatief is omdat ik geen specifieke details over het project heb.\n\n1. Gezamenlijke risicobeoordeling: Voor dit specifieke project kan het nuttig zijn om gezamenlijk een risicobeoordeling uit te voeren. Dit kan helpen om potentiële problemen te identificeren en plannen te maken om deze aan te pakken, wat kan bijdragen aan de onderhoudsbehoefte van het areaal.\n\n2. Technologie gebruiken: Afhankelijk van de aard van het project, kan het gebruik van bepaalde technologieën, zoals geavanceerde monitoring- en onderhoudssystemen, helpen om de onderhoudsbehoefte te verminderen en de samenwerking tussen de opdrachtgever en de opdrachtnemer te verbeteren.\n\n3. Regelmatige sitebezoeken: Voor dit project kan het nuttig zijn om regelmatige sitebezoeken te organiseren. Dit kan zowel de opdrachtgever als de opdrachtnemer een beter inzicht geven in de voortgang van het project en eventuele problemen die zich vo

In [16]:
agent.run(input="Kun je aangeven waaruit blijkt dat gezamenlijke besluitvorming als maatregel hoor gewaardeerd wordt door rijkswaterstaat?")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Rijkswaterstaat waardeert gezamenlijke besluitvorming omdat het bijdraagt aan een betere samenwerking en een meer inclusieve aanpak van projectmanagement. Het stelt alle betrokken partijen in staat om hun inzichten en expertise in te brengen, wat kan leiden tot betere beslissingen. Bovendien kan gezamenlijke besluitvorming helpen om misverstanden en conflicten te voorkomen, omdat iedereen betrokken is bij het proces en begrijpt hoe en waarom bepaalde beslissingen worden genomen. Dit is echter een algemene observatie en de specifieke waardering kan variëren afhankelijk van het project en de betrokken personen.

> Finished chain.


'Rijkswaterstaat waardeert gezamenlijke besluitvorming omdat het bijdraagt aan een betere samenwerking en een meer inclusieve aanpak van projectmanagement. Het stelt alle betrokken partijen in staat om hun inzichten en expertise in te brengen, wat kan leiden tot betere beslissingen. Bovendien kan gezamenlijke besluitvorming helpen om misverstanden en conflicten te voorkomen, omdat iedereen betrokken is bij het proces en begrijpt hoe en waarom bepaalde beslissingen worden genomen. Dit is echter een algemene observatie en de specifieke waardering kan variëren afhankelijk van het project en de betrokken personen.'

In [17]:
agent.run(input="Wat is het profiel van de gemiddelde besluitvormer van rijkswaterstaat, en hoe zouden we die op dit punt kunnen overtuigen? Speculeren mag, zo lang je het expliciet aangeeft.")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Het profiel van de gemiddelde besluitvormer bij Rijkswaterstaat kan variëren, maar over het algemeen zijn ze hoogopgeleid, met een achtergrond in techniek, projectmanagement of een gerelateerd veld. Ze hebben vaak jarenlange ervaring in hun vakgebied en hebben een diepgaand begrip van de complexiteit en uitdagingen van infrastructuurprojecten. Ze waarderen duidelijke, feitelijke informatie en rationele argumenten.

Om hen te overtuigen van het belang van gezamenlijke besluitvorming, zou je kunnen benadrukken hoe het kan bijdragen aan het succes van het project. Je zou kunnen wijzen op specifieke voorbeelden van projecten waar gezamenlijke besluitvorming heeft geleid tot betere resultaten. Je zou ook kunnen benadrukken hoe het kan helpen om problemen vroegtijdig te identificeren en aan te pakken, wat kan leiden tot kostenbesparingen op de lange termijn.

Daarnaast zou je kunnen benadrukken hoe gezamenlijke besluitvormin

'Het profiel van de gemiddelde besluitvormer bij Rijkswaterstaat kan variëren, maar over het algemeen zijn ze hoogopgeleid, met een achtergrond in techniek, projectmanagement of een gerelateerd veld. Ze hebben vaak jarenlange ervaring in hun vakgebied en hebben een diepgaand begrip van de complexiteit en uitdagingen van infrastructuurprojecten. Ze waarderen duidelijke, feitelijke informatie en rationele argumenten.\n\nOm hen te overtuigen van het belang van gezamenlijke besluitvorming, zou je kunnen benadrukken hoe het kan bijdragen aan het succes van het project. Je zou kunnen wijzen op specifieke voorbeelden van projecten waar gezamenlijke besluitvorming heeft geleid tot betere resultaten. Je zou ook kunnen benadrukken hoe het kan helpen om problemen vroegtijdig te identificeren en aan te pakken, wat kan leiden tot kostenbesparingen op de lange termijn.\n\nDaarnaast zou je kunnen benadrukken hoe gezamenlijke besluitvorming kan bijdragen aan een betere samenwerking en communicatie tus

In [20]:
agent.run(input="de aanbesteder verwacht een invulling ten aanzien van: 1. Het gezamenlijk toepassen van operationeel Assetmanagement op onderhoud- en inspectieregimes.  2. Het in samenwerking komen tot een hogere trede (voor beide partijen) op de veiligheidscultuurladder. 3. Realiseren van een soepele transitie naar een OMS in RWS beheer. 4.Het creëren van een robuust samenwerkingsklimaat waarin relationele aspecten zoals vertrouwen, persoonlijkheden, competenties, cultuur, belangen en openheid centraal staan. Hoe zou op basis van deze punten de optimale samenwerking tussen OG en ON kunnen invullen? Wees specifiek en speculeer, maar benoem dat expliciet.")



> Entering new  chain...


OutputParserException: Could not parse LLM output: `Om de optimale samenwerking tussen de Opdrachtgever (OG) en de Opdrachtnemer (ON) te realiseren op basis van de genoemde punten, zou je de volgende stappen kunnen overwegen. Let op, dit is speculatief en gebaseerd op algemene observaties:

1. Gezamenlijk toepassen van operationeel Assetmanagement: Dit kan worden bereikt door regelmatige bijeenkomsten te organiseren waarin beide partijen hun inzichten en expertise delen over onderhouds- en inspectieregimes. Het gebruik van gezamenlijke tools en platforms voor assetmanagement kan ook nuttig zijn om ervoor te zorgen dat alle relevante informatie gemakkelijk toegankelijk is voor beide partijen.

2. Samenwerken aan een hogere trede op de veiligheidscultuurladder: Dit kan worden bereikt door gezamenlijke veiligheidstrainingen en workshops te organiseren. Het is ook belangrijk om een open dialoog te hebben over veiligheidskwesties en om ervoor te zorgen dat alle teamleden zich comfortabel vo

In [23]:
text = agent.run("Beschrijf wat de meest succesvolle schrijfstijl is om dit project binnen te krijgen.")



> Entering new  chain...


OutputParserException: Could not parse LLM output: `Het schrijven van een succesvol projectvoorstel voor Rijkswaterstaat vereist een duidelijke, beknopte en overtuigende schrijfstijl. Hier zijn enkele richtlijnen:

1. Duidelijk en beknopt: Gebruik eenvoudige en duidelijke taal. Vermijd jargon waar mogelijk en leg technische termen uit. Zorg ervoor dat uw punten gemakkelijk te begrijpen zijn.

2. Feitelijk en onderbouwd: Ondersteun uw argumenten met feiten en gegevens. Dit toont aan dat u uw onderzoek heeft gedaan en dat uw voorstellen realistisch en haalbaar zijn.

3. Overtuigend: Uw doel is om de besluitvormers te overtuigen van de waarde van uw project. Benadruk de voordelen en de positieve impact van uw project.

4. Gericht op de behoeften van Rijkswaterstaat: Toon aan dat u de behoeften en doelen van Rijkswaterstaat begrijpt. Leg uit hoe uw project aan deze behoeften voldoet en bijdraagt aan de doelen van Rijkswaterstaat.

5. Professioneel: Zorg ervoor dat uw voorstel professioneel

In [24]:
# show text in markdown
from IPython.display import Markdown
Markdown(text)

NameError: name 'text' is not defined